# Poincare Boundary Condition

Most of the tutorials use last closed flux surface (LCFS) as the boundary condition for 3D equilibrium. This tutorial will cover another way to define boundary condition using DESC.

In [ ]:
import sys
import os

sys.path.insert(0, os.path.abspath("."))
sys.path.append(os.path.abspath("../../../"))

If you have access to a GPU, uncomment the following two lines. 

In [ ]:
from desc import set_device
set_device("gpu")

As mentioned in [DESC Documentation on performance tips](https://desc-docs.readthedocs.io/en/latest/performance_tips.html), one can use compilation cache directory to reduce the compilation overhead time. Note: One needs to create `jax-caches` folder manually.

In [ ]:
# import jax

# jax.config.update("jax_compilation_cache_dir", "../jax-caches")
# jax.config.update("jax_persistent_cache_min_entry_size_bytes", -1)
# jax.config.update("jax_persistent_cache_min_compile_time_secs", 0)

In [ ]:
%matplotlib inline
import numpy as np

from desc.io import load
from desc.equilibrium import EquilibriaFamily, Equilibrium
from desc.continuation import solve_continuation_automatic
from desc.optimize import Optimizer
from desc.grid import LinearGrid
from desc.objectives import (
    ObjectiveFunction,
    ForceBalance,
    FixCurrent,
    FixSectionLambda,
    FixSectionR,
    FixSectionZ,
    FixBoundaryR,
    FixBoundaryZ,
    FixPressure,
    FixPsi,
    FixIota,
    QuasisymmetryTwoTerm,
    AspectRatio,
    Elongation,
    Volume,
    RotationalTransform,
    get_fixed_xsection_constraints,
    get_fixed_boundary_constraints,
)
from desc.examples import get
from desc.plotting import *
from desc.geometry import ZernikeRZToroidalSection, FourierRZToroidalSurface
from desc.backend import print_backend_info
from desc.compat import rotate_zeta

import plotly.express as px
import plotly.io as pio

# This ensures Plotly output works in multiple places:
# plotly_mimetype: VS Code notebook UI
# notebook: "Jupyter: Export to HTML" command in VS Code
# See https://plotly.com/python/renderers/#multiple-renderers
pio.renderers.default = "plotly_mimetype+notebook"

print_backend_info()

## Optimization with Poincare Solve

During equilibrium optimization, if a user gives `ForceBalance` as constraint, DESC automatically uses `proximal-lsq-exact` which solve the equilibrium at each optimization step to preserve the MHD force balance. This internal equilibrium solve use fixed-boundary constraint by default. However, this can be changed to fixed-Poincare section solve, if user adds one of the `FixSection...` constraints instead of `FixBoundary...` constraints. Let's see an example!

We will conduct a similar optimization as shown in `desc.examples.precise_QH.py`.

In [ ]:
def set_poincare_equilibrium(eq):
    eq_poincare = Equilibrium(
        xsection=eq.get_surface_at(zeta=0),
        pressure=eq.pressure,
        iota=eq.iota,
        Psi=eq.Psi,  # flux (in Webers) within the last closed flux surface
        NFP=eq.NFP,  # number of field periods
        L=eq.L,  # radial spectral resolution
        M=eq.M,  # poloidal spectral resolution
        N=eq.N,  # toroidal spectral resolution
        L_grid=eq.L_grid,  # real space radial resolution, slightly oversampled
        M_grid=eq.M_grid,  # real space poloidal resolution, slightly oversampled
        N_grid=eq.N_grid,  # real space toroidal resolution
        sym=eq.sym,  # explicitly enforce stellarator symmetry
        spectral_indexing=eq._spectral_indexing,
    )

    eq_poincare.change_resolution(eq.L, eq.M, eq.N)
    eq_poincare.axis = eq_poincare.get_axis()
    eq_poincare.surface = eq_poincare.get_surface_at(rho=1)
    return eq_poincare

In [ ]:
try:
    eq = load("poincare_precise_QA_initial_eq_using_v16_updated.h5")
    eq.xsection = eq.get_surface_at(zeta=0)
    eq.surface = eq.get_surface_at(rho=1)
    raise FileNotFoundError()
except FileNotFoundError:
    eq = load("../../../desc/examples/precise_QA_output.h5")[0]
    eq.change_resolution(L=12, M=12, N=6, L_grid=24, M_grid=24, N_grid=12)
    eq.solve(maxiter=500, verbose=3, ftol=1e-3)
    eq.xsection = eq.get_surface_at(zeta=0)
    eq.surface = eq.get_surface_at(rho=1)
    constraints = get_fixed_xsection_constraints(eq=eq, fix_lambda=True)
    objective = ObjectiveFunction(ForceBalance(eq))

    # before optimization make sure that the initial equilibrium
    # is in force balance in terms of poincare constraints
    eq.solve(
        verbose=3,
        objective=objective,
        constraints=constraints,
        maxiter=1000,
        ftol=1e-3,
    );
    eq.xsection = eq.get_surface_at(zeta=0)
    eq.surface = eq.get_surface_at(rho=1)
    # eq.save("poincare_precise_QA_initial_eq_using_v16_updated.h5")

In [ ]:
plot_section(eq, "|F|_normalized", log=True, phi=3);
plot_surfaces(eq, phi=3);

In [ ]:
eqfam = EquilibriaFamily(eq)
eq00 = get("precise_QA")
V = eq.compute("V")["V"]
Vorg = eq00.compute("V")["V"]

# grid for computing quasisymmetry objective
grid = LinearGrid(M=eq.M, N=eq.N, NFP=eq.NFP, rho=np.array([0.6, 0.8, 1.0]), sym=True)

In [ ]:
# eqfam = load("./eqfam_poincare_optimize_QA_wqs1.0_war1.0_wvol50.0_wiota20.0.h5")
# eq00 = get("precise_QA")
# Vorg = eq00.compute("V")["V"]

# # grid for computing quasisymmetry objective
# grid = LinearGrid(
#     M=eq00.M, N=eq00.N, NFP=eq00.NFP, rho=np.array([0.6, 0.8, 1.0]), sym=True
# )

In [ ]:
from desc.objectives.normalization import compute_scaling_factors
compute_scaling_factors(eq)

In [ ]:
print(f"Initial volume: {V:.6f}, Volume of precise QA: {Vorg:.6f}")

In [ ]:
w_qs = 1
w_ar = 10
w_vol = 50
w_iota = 10

def run_step(n, eqfam, ftol=1e-2, **kwargs):
    objective = ObjectiveFunction(
        (
            QuasisymmetryTwoTerm(
                eq=eqfam[-1],
                helicity=(1, 0),
                grid=grid,
                normalize=False,
                weight=w_qs,
            ),
            AspectRatio(eq=eqfam[-1], target=6, weight=w_ar, normalize=False),
            Volume(
                eq=eqfam[-1], target=Vorg, weight=w_vol, normalize=False
            ), 
            RotationalTransform(
                eq=eqfam[-1], target=0.42, weight=w_iota, normalize=False
            ),
        ),
    )
    # modes to fix
    bc_surf = eqfam[-1].xsection
    R_modes = bc_surf.R_basis.modes[np.max(np.abs(bc_surf.R_basis.modes), 1) > n, :]
    Z_modes = bc_surf.Z_basis.modes[np.max(np.abs(bc_surf.Z_basis.modes), 1) > n, :]
    constraints = (
        ForceBalance(eq=eqfam[-1]),
        FixSectionR(eq=eqfam[-1], modes=R_modes),
        FixSectionZ(eq=eqfam[-1], modes=Z_modes),
        FixPressure(eq=eqfam[-1]),
        FixCurrent(eq=eqfam[-1]),
        FixPsi(eq=eqfam[-1]),
    )

    optimizer = Optimizer("proximal-lsq-exact")
    eq_new, _ = eqfam[-1].optimize(
        objective=objective,
        constraints=constraints,
        optimizer=optimizer,
        maxiter=250,
        verbose=3,
        ftol=ftol,
        copy=True,
        options={
            "perturb_options": {"verbose": 0},
            "solve_options": {"verbose": 0, "ftol": 1e-3, "maxiter": 250},
            **kwargs,
        },
    )
    # to make sure the surfaces are updated properly
    eq_new.xsection = eq_new.get_surface_at(zeta=0)
    eq_new.surface = eq_new.get_surface_at(rho=1)
    eqfam.append(eq_new)
    return eqfam

In [ ]:
eqfam = run_step(12, eqfam, ftol=1e-3)
plot_boozer_surface(eqfam[-1]);

In [ ]:
plot_comparison(eqs=[eqfam[0], eqfam[-1]], labels=["Initial", "Final"]);

In [ ]:
plot_3d(eqfam[-1], "|B|")

In [ ]:
eq00 = get("precise_QA")
fig, ax = plot_1d(eqfam[-1], "iota", label="Poincare Optimized", color="blue")
plot_1d(eq00, "iota", ax=ax, label="precise_QA", color="red")

In [ ]:
fig = plot_3d(eqfam[-1], "|B|")
plot_3d(eq00, "|B|", fig=fig)

In [ ]:
eq_rotated = eqfam[-1].copy()
eq_rotated = rotate_zeta(eq_rotated, np.pi / 2)
fig = plot_3d(eq_rotated, "|B|")
plot_3d(eq00, "|B|", fig=fig, alpha=0.3)

In [ ]:
plot_comparison(
    eqs=[eqfam[0], eq_rotated, eq00],
    labels=["Initial", "Poincare", "precise_QA"],
    color=["black", "blue", "red"],
);

In [ ]:
levels = np.logspace(-6, 0, 50)
plot_section(eq_rotated, "|F|_normalized", log=True, phi=3, levels=levels)
plot_section(eq00, "|F|_normalized", log=True, phi=3, levels=levels);

In [ ]:
plot_boozer_modes(eq00, helicity=(1, 0))
plot_boozer_modes(eqfam[-1], helicity=(1, 0));

In [ ]:
plot_qs_error(eqfam[-1], helicity=(1, 0))
plot_qs_error(eq00, helicity=(1, 0));

In [ ]:
plot_boozer_surface(eqfam[-1], rho=0.7);